# Data Journalism Lesson 15: Stacked bar charts

Extending the bar chart to add components.

In [ ]:
import warnings
from IPython.core.interactiveshell import InteractiveShell

# Keep hold of the real method
_orig_should_run = InteractiveShell.should_run_async

# Wrap it so that any DeprecationWarning it emits is silenced
def should_run_async(self, code, *args, **kwargs):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=DeprecationWarning)
        return _orig_should_run(self, code, *args, **kwargs)

# Apply the monkey‑patch
InteractiveShell.should_run_async = should_run_async

In [ ]:
import micropip
await micropip.install('plotly')
await micropip.install("nbformat>=4.2.0")

In [ ]:
# Setup code for the notebook
import pandas as pd
from IPython.display import display, HTML

# Helper functions for grading exercises
def display_feedback(correct, message_correct, message_incorrect):
    if correct:
        display(HTML(f'<div style="background-color: #dff0d8; padding: 10px; border-radius: 5px;"><strong>Correct!</strong> {message_correct}</div>'))
    else:
        display(HTML(f'<div style="background-color: #f2dede; padding: 10px; border-radius: 5px;"><strong>Not quite!</strong> {message_incorrect}</div>'))

def check_read_csv(df, expected_rows, file_description):
    correct = isinstance(df, pd.DataFrame) and (expected_rows is None or len(df) == expected_rows)
    message_correct = f'Great work! You imported the {file_description} data.'
    message_incorrect = f'Check the file path or the read_csv function for the {file_description} data.'
    if isinstance(df, pd.DataFrame) and expected_rows is not None and len(df) != expected_rows:
        message_incorrect += f' Expected {expected_rows} rows, but got {len(df)}.'
    display_feedback(correct, message_correct, message_incorrect)

def check_df_operation(result_df, expected_df, operation_name, check_columns=True, check_values=True, rtol=1e-3):
    correct = True
    message = ""
    if not isinstance(result_df, pd.DataFrame):
        correct = False
        message = f'The result of {operation_name} should be a DataFrame.'
    else:
        try:
            pd.testing.assert_frame_equal(result_df.reset_index(drop=True), expected_df.reset_index(drop=True), check_dtype=False, rtol=rtol, check_like=True)
            message = f'Correctly performed the {operation_name} operation!'
        except AssertionError as e:
            correct = False
            message = f'The {operation_name} result is not as expected. Details: {e}'
    display_feedback(correct, message, message if not correct else f'Something is off with the {operation_name}. Review the steps.')

def check_plot_params(params, expected_params, plot_name):
    correct = True
    messages = []
    for p_name, p_val in expected_params.items():
        if params.get(p_name) == p_val:
            messages.append(f'Correct {p_name} for {plot_name}.')
        else:
            correct = False
            messages.append(f'Incorrect {p_name} for {plot_name}. Expected \'{p_val}\', got \'{params.get(p_name)}\'.')
    final_message_correct = f'Plot parameters for {plot_name} are correct!'
    final_message_incorrect = ' '.join(messages)
    display_feedback(correct, final_message_correct, final_message_incorrect)

In [ ]:
# Load required data and prepare expected results for checks
state_name = "Minnesota"
state_abbr = "MN" # Used for county language lookup, though simplified here

# Load investments data
investments_file_path = f"../_static/rural-grants/{state_name.lower().replace(' ', '-')}.csv"
investments_df = pd.read_csv(investments_file_path)
investments_rows_expected = len(investments_df) # Store for checking later if needed

# Load census estimates data
estimates_file_path = f"../_static/census-estimates/{state_name.lower().replace(' ', '-')}.csv"
estimates23_df_raw = pd.read_csv(estimates_file_path)
estimates23_df = estimates23_df_raw.copy()
estimates23_df['GEOID'] = estimates23_df['STATE'].astype(str).str.zfill(2) + estimates23_df['COUNTY'].astype(str).str.zfill(3)
estimates23_df = estimates23_df[['GEOID', 'POPESTIMATE2023', 'CTYNAME']]
estimates_rows_expected = len(estimates23_df_raw) # Store for checking later

# Define county language (simplified)
county_language_singular = "county"
county_language_plural = "counties"

# Pre-calculate expected results for exercises
impact_df_expected = investments_df.groupby(['county', 'county_fips'], as_index=False).agg(
    total_investments=('number_of_investments', 'sum'),
    total_dollars=('investment_dollars', 'sum')
)
impact_df_expected['county_fips'] = impact_df_expected['county_fips'].astype(str) # Ensure type for merge
estimates23_df['GEOID'] = estimates23_df['GEOID'].astype(str) # Ensure type for merge
impact_df_expected = pd.merge(impact_df_expected, estimates23_df, left_on='county_fips', right_on='GEOID')
impact_df_expected['investment_dollars_per_person'] = impact_df_expected['total_dollars'] / impact_df_expected['POPESTIMATE2023']
impact_df_expected = impact_df_expected.sort_values('investment_dollars_per_person', ascending=False)
impact_df_expected = impact_df_expected.nlargest(15, 'investment_dollars_per_person').reset_index(drop=True)

stacked_df_expected = investments_df.groupby(['county', 'program_area'], as_index=False).agg(
    program_dollars=('investment_dollars', 'sum')
)
stacked_df_expected = pd.merge(stacked_df_expected, impact_df_expected[['county', 'total_dollars', 'county_fips', 'POPESTIMATE2023', 'investment_dollars_per_person']], on='county').reset_index(drop=True)

# For plot reordering, get county order based on total_dollars from the top 15 impacted counties
if not stacked_df_expected.empty:
    county_order_df_expected = stacked_df_expected[['county', 'total_dollars']].drop_duplicates().sort_values('total_dollars', ascending=True)
    county_order_for_plot_expected = county_order_df_expected['county'].tolist()
else:
    county_order_for_plot_expected = []

In [ ]:
# Glue variables for use in markdown
from myst_nb import glue

glue("state_full_name", state_name, display=False)
glue("investments_rows", f"{investments_rows_expected: ,}", display=False)
glue("county_language_singular", county_language_singular, display=False)
glue("county_language_plural", county_language_plural, display=False)

# Add more glue variables as needed from the Rmd's setup
if not impact_df_expected.empty:
    # Example: Top county by investment per person (from impact_df_expected)
    top_impact_county = impact_df_expected.iloc[0]['county']
    glue("top_impact_county_name", top_impact_county, display=False)
else:
    glue("top_impact_county_name", "N/A", display=False)

## The Goal

In this lesson, you'll learn how to create stacked bar charts, a powerful visualization for showing both the total magnitude and composition of data across categories. By the end of this tutorial, you'll understand when to use stacked bar charts, how to prepare data in the correct format, and how to construct these charts using Plotly. You'll practice these skills using real USDA rural investment data, gaining hands-on experience in visualizing how different program areas contribute to total investments across counties. This ability to show both overall amounts and their component parts will be invaluable for presenting complex, multi-category data in your journalism projects.

## Why Visualize Data?

If you study data science long enough, you're going to come across the name John W. Tukey. Tukey was a statistician at Princeton, where he got his PhD before World War II and returned to after the war. His biography is fascinating. Among the algorithms and statistical tests he's credited with creating, he also first published the word *software*. For 20 years, Tukey helped design NBC News' polls to predict elections. He consulted widely and wrote extensively.

One thing he is most known for, in the data science world, is the concept of Exploratory Data Analysis, an idea data journalism owes a lot of debts to. What is EDA? The Preface of Tukey's book Exploratory Data Analysis, written in 1977 starts with this:

> This book is based on an important principle: It is important to understand what you CAN DO before you learn to measure how WELL you seem to have DONE it.

In the next paragraph, he goes further:

> This book is about exploratory data analysis, about looking at data to see what it seems to say. It concentrates on simple arithmetic and easy-to-draw pictures. It regards whatever appearances we have recognized as partial descriptions, and tries to look beneath them for new insights. 

That seems like a pretty good definition of data journalism. 

Exploratory Data Analysis, the book, is not for the faint of heart. Most of the foundational texts of data science come with some flourish -- a little philosophy and some explanation to go with your technique. Not Tukey. His style is direct, brutally so. It moves with a purpose, from concept, to example, to problems, to review questions, with very little fat. One person who saw him lecture described the words in his lecture as "not many, like overweight parcels, delivered at a slow unfaltering pace."

Exploratory Data Analysis is also a monument of its time. To read it today, you have to understand that computers were rare, exceedingly expensive, and monstrously complicated. Most people in the world had never seen one in person in 1977. So everything he describes is all by hand.

"The first thing we need to do is separate, in our mind, what it takes to make plotting easy from what it takes to make plotting effective," Tukey wrote. "The lines ruled on graph paper help to make plotting easy, but they do not make plotting effective for seeing what is going on -- instead they get in the way of seeing what we ought to see."

Graph paper. Most students today would struggle to find graph paper. Tukey was a huge fan (he even drops his favorite graph paper company in the text). He even gives tips on how to save graph paper. 

"If we want to see what our plots ought to tell us, there is no substitute for the use of tracing paper (or acetate)," he wrote. "If we slip a well-printed sheet of graph paper just below the top sheet of a pad of tracing paper, we can plot on that top sheet of tracing paper almost as easily as if it were itself ruled. Then, when we have the points plotted, some boundary or reference lines drawn, and a few scale points ticked, we can take away the graph sheet and look at the points undisturbed by a grid. We often gain noticeably in insight by doing this. (And we have had to pay for a sheet of tracing paper rather than for a sheet of graph paper.)"

Today, computers are common, and the power to make charts is at your fingertips.

But make no mistake, what we are doing with computers here is exactly what Tukey advised students to do with Exploratory Data Analysis and paper. 

"We almost always want to look at numbers. We do not always have graph paper at hand," he wrote. 

"There is no excuse for failing to plot and look."

## The Basics

In the last chapter, we looked at the humble bar chart, which is very good at showing how much something is in relation to other things in the chart. It shows magnitude. Today, we're adding a wrinkle to the bar chart that is going to still show magnitude, but now it's going to show composition of that magnitude. It's called the stacked bar chart, and it's very good at showing not only how much of something there is compared to other things, but how much of the things that make up the whole there are. 

Let's use an example that should feel close to home: grades. It's different at every university, but most students take somewhere in the neighorhood of 40 classes. Some students -- like yours truly -- are bad students and end up taking a few more (again). Some students are on the ball and came in with college credit and don't have to take as many classes. But it's all somewhere in that neighborhood. 

With a stacked bar chart, you could look at not only how *many* classes each student took, but what was the grade breakdown they had in those classes. Imagine a bar, broken down into parts. Here's the A part, the B part, the C part, the D part and the F part. Your super nerds who need to go outside and touch some grass are going to have a very big A part and maybe very little of anything else.  Your average student will have some As, some Bs, maybe a C or two and not much else. Terrible students like your author will have some As, some Bs, some Cs, some Ds and a light smattering of Fs that had to be taken over again. What can I say? Ever meet someone on campus who really should have gone somewhere else and grown up a few years before trying to go to college? That was me. 

The point is, how every student gets to graduation day is different. The stacked bar will show that difference far better than a regular bar chart.

To make a stacked bar chart work, you need long data. Recall that we ran into this problem in the Census chapter, where we learned the difference between long data and wide data. To review, using our student grades example, wide data would be something like this:

|Student|As  |Bs  |Cs  |Ds  |Fs  |  
|-------|----|----|----|----|----|
|John   |15  |15  |10  |0   |0   |
|Jane   |25  |10  |5   |0   |0   |

Long data, on the other hand, would be like this:

|Student|Grade|Count|
|-------|-----|-----|
|John   |A    |15   |
|John   |B    |15   |
|John   |C    |10   |
|John   |D    |0    |
|John   |F    |0    |
|Jane   |A    |25   |
|Jane   |B    |10   |
|Jane   |C    |5    |
|Jane   |D    |0    |
|Jane   |F    |0    |

What Plotly is going to do is group each student together by their name, then create a bar chart of each grade count all on the same line. We'll tell it to make each segment a different color to separate them out. 

Instead of grades -- which I don't have -- let's use our USDA data from the bar charts lesson, and this time we're going to speed up the making of the data a smidge to get to it quicker.

First, we need libraries. Pandas for data manipulation and Plotly Express for plotting.

In [ ]:
import pandas as pd
import plotly.express as px

Now let's get the investments data.

In [ ]:
investments_df = pd.read_csv(f"../_static/rural-grants/minnesota.csv")
display(investments_df.head())

And now some (preprocessed) census data.

In [ ]:
estimates23_df = pd.read_csv("../_static/census-estimates/minnesota.csv")
estimates23_df['GEOID'] = estimates23_df['STATE'].astype(str).str.zfill(2) + estimates23_df['COUNTY'].astype(str).str.zfill(3)
estimates23_df = estimates23_df[['GEOID', 'POPESTIMATE2023', 'CTYNAME']]
estimates23_df = estimates23_df.rename(columns={'CTYNAME': 'county_name', 'POPESTIMATE2023': 'population_estimate'})
estimates23_df['GEOID'] = estimates23_df['GEOID'].astype(str) # Ensure type for merge
display(estimates23_df.head())

### Exercise 1: Making impact

Now, to speed things up, we're going to create a DataFrame called `impact_df` that is going to do everything we did to prepare our bar charts before, but in one block. We're going to total up investments in each {glue:text}`county_language_singular`, join the census data to it, create a new `investment_dollars_per_person` column using `assign`, and use `nlargest` to get the top 15 {glue:text}`county_language_plural` most impacted by these investments. It might *seem* like a lot of code, but it's all things you've already done, just stacked together instead of pulled apart into steps.

Fill in the blanks (`____`) below.

In [ ]:
impact_df = investments_df.groupby([____, ____], as_index=False).agg(
    total_investments=('number_of_investments', 'sum'),
    total_dollars=('investment_dollars', 'sum')
)
impact_df['county_fips'] = impact_df['county_fips'].astype(str)

impact_df = pd.merge(impact_df, estimates23_df, left_on="county_fips", right_on="GEOID")
impact_df = impact_df.assign(
    investment_dollars_per_person = lambda x: x[____] / x[____]
)
impact_df = impact_df.sort_values(____, ascending=False)
impact_df = impact_df.nlargest(____, ____).reset_index(drop=True)

display(impact_df.head())

Now, our data is currently just {glue:text}`county_language_singular` totals with a new measure of impact. And that's fine for bar charts, but for stacked bar charts we're looking at **categories** of our data within each {glue:text}`county_language_singular`. One thing you want to be careful about is the *number* of categories you include. Rule of thumb? About the max you want to look at is 5. Any more than that and you're going to have a lot of colors to manage and it's hard to interpret what is going on.

### Exercise 2: Two groups

Our original `investments_df` data has a column called `program_area` that we've looked at before in the filtering chapter. There, we looked at investments in housing. But here, since we're interested in all of the categories, it would be useful for us to create a summary of dollars in each {glue:text}`county_language_singular` AND each `program_area`. See if you can re-interpret how I said that into code.

In [ ]:
investments_by_program_area = investments_df.groupby([____, ____], as_index=False).agg(
    program_dollars=(____, 'sum')
)

display(investments_by_program_area.head())

And, notice, we have long data. Each `program_area` is one line. So a {glue:text}`county_language_singular` appears multiple times, but each `program_area` appears only once for that {glue:text}`county_language_singular` (if they have an investment in that particular area).

So what's left to do? Note we have long data for *every* {glue:text}`county_language_singular` in {glue:text}`state_full_name`. That is going to be way too much data. And we're interested in the top 15 by impact -- a number we *don't* have in our data. How do we get just the top 15, but by `program_area`?

Joining. Remember: we have a DataFrame called `impact_df` and it has stuff we need in it. Another thing to remember from the joining chapter -- when we use `pd.merge` with default `how='inner'`, it keeps all rows that match from **both** DataFrames. Meaning if we use an inner merge, we'll get all rows from our `program_area` group by work *and* a row that matches the {glue:text}`county_language_singular` name from our top 15. We *won't* get places that *aren't* in the top 15.

### Exercise 3: Joining

While we're at this, we're going to store this in a new DataFrame called `stacked_df`.

In [ ]:
# We'll use investments_by_program_area from the previous exercise
# And impact_df for the top 15 counties info
stacked_df = pd.merge(investments_by_program_area, 
                               impact_df, 
                               on=____).reset_index(drop=True)

display(stacked_df.head())

We'll take a look at the results in the next section, but you're ready to move forward. You've got long data, broken down by categories, for a small enough set of data that it will make a useful graphic.

## The Stacked Bar

To make a stacked bar chart, we need three things: A grouping (in our case, a `county`), a number to give the bars size, and a category to give the bars color (and to separate the components). 

Let's take a look at our data (`stacked_df`).

In [ ]:
display(stacked_df.head())

More often than not with stacked bar charts, what you need to make each bar and to make the colors comes from your group by work. So just like last time, each bar is going to be a `county`. We're looking at the `program_dollars` this time to give the bars size, and each `program_dollar` comes from a `program_area`. 

How do we give each segment a new color? We're going to add a new thing to our bar chart called the `color` argument in `plotly.express.bar`. It does what you think -- fills in the bar with a color for each `program_area` in that {glue:text}`county_language_singular`.

### Exercise 4: The stacked bar

In [ ]:
fig1 = px.bar(
    data_frame=stacked_df,
    x=_____,
    y=_____,
    color=______
)
fig1.show()

And just like that, we've made a stacked bar chart. But just like our first bar chart, it needs help to be readable. What we did last time works here too, but we're going to add one wrinkle. Last time, we reordered by the same number that determined bar height. In this case, we want the bars to be in order of `total_dollars` (overall impact for the {glue:text}`county_language_singular`), not `program_dollars` (segment size). Think of it this way -- the `total_dollars` determines the total size of the conceptual bar (though Plotly stacks `program_dollars`). The `program_dollars` determines the size of each segment. And we'll use `orientation='h'` and swap x/y to make it more readable (horizontal bars).

### Exercise 5: Reorder and Flipping (Horizontal Bars)

In [ ]:
fig2 = px.bar(
    data_frame=stacked_df.sort_values(by=_____, ascending=True),
    x=_____,
    y=_____,
    color=______,
    orientation='h',
)
fig2.show()

Now, just like our mythical students and their quest to get to graduation day, each {glue:text}`county_language_singular` in {glue:text}`state_full_name` gets to the total dollars of investments from the USDA in a different way. Major programs include community centers and business programs. Look at your {glue:text}`county_language_plural` in your chart and interpret how the top places built their total investments. What program areas were the focus in those places? How are they different from each other? That's what stacked bar charts invite you to do.

## The Recap

Throughout this lesson, you've mastered the creation of stacked bar charts, a versatile tool for visualizing both totals and their component parts. You've learned to prepare data in the necessary "long" format, combine datasets to focus on the most relevant information, and use Plotly to construct informative stacked bar charts. Remember, stacked bar charts are particularly effective when you need to show both overall magnitudes and the composition of those totals across categories.

## Terms to Know

- **Stacked bar chart**: A type of bar chart where bars are divided into segments, each representing a subcategory of the total.
- **Long data**: A data format where each row represents a single observation, with categories spread across multiple rows rather than columns.
- **Wide data**: A data format where each row represents a complete set of observations, with categories spread across multiple columns.
- **`plotly.express.bar()`**: A Plotly function used to create bar charts, including stacked bar charts when `color` is specified and `barmode='stack'` (often default).
- **`color` argument**: An argument in `px.bar()` used to specify the variable that determines the color of bar segments in a stacked bar chart.
- **`orientation='h'`**: An argument in `px.bar()` to create horizontal bar charts.